# Заливка данных

In [1]:
import pandas as pd
import numpy as np

### Чтение csv-файла

In [2]:
df = pd.read_csv('german-credit.csv')

In [3]:
pd.set_option('display.float_format', '{:,.2f}'.format)
# разделитель ',' и два знака после запятой у чисел с плавающей точкой

In [4]:
df.head().style.format(thousands=',')
# разделитель ',' для всех чисел

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,default
0,0,67,male,2,own,nan,little,"1,169",6,radio/TV,0
1,1,22,female,2,own,little,moderate,"5,951",48,radio/TV,1
2,2,49,male,1,own,little,nan,"2,096",12,education,0
3,3,45,male,2,free,little,little,"7,882",42,furniture/equipment,0
4,4,53,male,2,free,little,little,"4,870",24,car,1


# Предобработка

### Удаление столбца

In [5]:
df.drop('Unnamed: 0', axis=1, inplace=True)

### Переименование столбцов

In [6]:
df = df.rename(columns=lambda x: x.lower().replace(' ', '_'))

In [7]:
df.head()

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default
0,67,male,2,own,NaN,little,1169,6,radio/TV,0
1,22,female,2,own,little,moderate,5951,48,radio/TV,1
2,49,male,1,own,little,NaN,2096,12,education,0
3,45,male,2,free,little,little,7882,42,furniture/equipment,0
4,53,male,2,free,little,little,4870,24,car,1


In [13]:
t = df.groupby(['housing', 'sex'])['age'].agg(['count', 'mean', 'median', 'min', 'max'])

In [16]:
t.head()

count  mean  median  min  max
housing sex                                  
free    female     19 48.11   48.00   24   74
        male       89 42.90   42.00   22   75
own     female    196 33.67   30.00   20   75
        male      517 36.32   34.00   20   74
rent    female     95 27.96   24.00   19   59

In [15]:
t['count'].sum()

1000

### Агрегирование

#### встроенные функции агрегирования

In [42]:
(df.groupby(['checking_account', 'sex'])
    [['credit_amount', 'default']]
    .agg({
        'default': ['mean', 'count'],
        'credit_amount': ['count', 'median']
    },))

default       credit_amount         
                           mean count         count   median
checking_account sex                                        
little           female    0.52    88            88 1,838.50
                 male      0.48   186           186 2,589.50
moderate         female    0.48    86            86 2,651.00
                 male      0.35   183           183 2,520.00
rich             female    0.20    20            20 1,400.50
                 male      0.23    43            43 1,925.00

базовые мат функции: включая количество, сумму (sum), среднее значение (mean), медианное значение (median),минимум (minimum), максимум (maximum), стандартное отклонение (standard deviation), дисперсию (variance), среднее абсолютное отклонение (mean absolute deviation) и произведение (product)

In [35]:
agg_func_math = {
    'credit_amount': ['count', 'sum', 'mean', 'median', 'min', 'max', 'std', 'var', 'mad', 'prod']
}
# задаем список функций по определенному столбцу

In [36]:
df.groupby('checking_account', dropna=False).agg(agg_func_math).round(2)
# учитываем строки с NaN: dropna=False

credit_amount                                         \
                         count      sum     mean   median  min    max   
checking_account                                                        
little                     274   870010 3,175.22 2,353.50  338  15857   
moderate                   269  1029614 3,827.56 2,622.00  276  18424   
rich                        63   137192 2,177.65 1,881.00  392   6289   
NaN                        394  1234442 3,133.10 2,248.00  250  15653   

                                                  \
                      std           var      mad   
checking_account                                   
little           2,636.38  6,950,520.91 1,919.64   
moderate         3,465.20 12,007,639.40 2,595.81   
rich             1,343.19  1,804,156.20 1,070.15   
NaN              2,554.16  6,523,755.47 1,878.22   

                                                                     
                                                               prod  
checking_account                                                     
little                                                          inf  
moderate                                                        inf  
rich             18,952,449,088,117,196,686,161,985,600,512,499,...  
NaN                                                             inf

In [37]:
df['checking_account'] = df['checking_account'].fillna('no_info')
# переименовываем NaN: fillna('no_info')

In [56]:
df.groupby('checking_account').agg(agg_func_math).round(2)

credit_amount                                                 \
                         count      sum     mean  median  min    max      std   
checking_account                                                                
little                     274   870010  3175.22  2353.5  338  15857  2636.38   
moderate                   269  1029614  3827.56  2622.0  276  18424  3465.20   
no_info                    394  1234442  3133.10  2248.0  250  15653  2554.16   
rich                        63   137192  2177.65  1881.0  392   6289  1343.19   

                                                       
                          var      mad           prod  
checking_account                                       
little             6950520.91  1919.64            inf  
moderate          12007639.40  2595.81            inf  
no_info            6523755.47  1878.22            inf  
rich               1804156.20  1070.15  1.895245e+205

In [41]:
agg_func_describe = {'credit_amount': ['describe']}
# describe вызывается описательная статистика

In [69]:
df.groupby('checking_account').agg(agg_func_describe).round(2)

credit_amount                                            \
                      describe                                             
                         count     mean      std    min      25%     50%   
checking_account                                                           
little                   274.0  3175.22  2636.38  338.0  1353.50  2353.5   
moderate                 269.0  3827.56  3465.20  276.0  1391.00  2622.0   
no_info                  394.0  3133.10  2554.16  250.0  1414.25  2248.0   
rich                      63.0  2177.65  1343.19  392.0  1275.00  1881.0   

                                   
                                   
                     75%      max  
checking_account                   
little            3954.0  15857.0  
moderate          5084.0  18424.0  
no_info           3804.0  15653.0  
rich              2969.5   6289.0

In [30]:
df.pivot_table(
    index='checking_account',
    columns='sex',
    values=['default', 'credit_amount'],
    aggfunc=['count','mean'],
    margins=True
)

count                                      mean  \
                 credit_amount           default           credit_amount   
sex                     female male  All  female male  All        female   
checking_account                                                           
little                      88  186  274      88  186  274      2,564.22   
moderate                    86  183  269      86  183  269      3,720.03   
rich                        20   43   63      20   43   63      1,616.05   
All                        194  412  606     194  412  606      2,978.84   

                                                      
                                   default            
sex                  male      All  female male  All  
checking_account                                      
little           3,464.30 3,175.22    0.52 0.48 0.49  
moderate         3,878.09 3,827.56    0.48 0.35 0.39  
rich             2,438.86 2,177.65    0.20 0.23 0.22  
All              3,541.07 3,361.08    0.47 0.40 0.42

In [39]:
df.groupby('checking_account')['default'].agg(['count', 'mean'])

,count,mean
checking_account,,
little,274,0.49
moderate,269,0.39
no_info,394,0.12
rich,63,0.22


In [42]:
(df.groupby('checking_account')['default']
.agg(['count', 'mean'])
.sort_values('count', ascending=False)
.style
.format({
    'mean': '{:,.2}'.format
})
.applymap(
    lambda x: 'background-color : green' if x > 0.3 else (
        'background-color : blue' if 0.2 < x <= 0.3 else ''),
    subset=['mean']
)
    )

,count,mean
checking_account,,
no_info,394,0.12
little,274,0.49
moderate,269,0.39
rich,63,0.22


In [43]:
t = df.pivot_table(
    index='checking_account',
    columns='sex',
    values=['default', 'credit_amount'],
    aggfunc=['count','mean'],
    margins=True
)

In [44]:
t.columns = ['_'.join(col).strip() for col in t.columns.values]

In [45]:
t

,count_credit_amount_female,count_credit_amount_male,count_credit_amount_All,count_default_female,count_default_male,count_default_All,mean_credit_amount_female,mean_credit_amount_male,mean_credit_amount_All,mean_default_female,mean_default_male,mean_default_All
checking_account,,,,,,,,,,,,
little,88,186,274,88,186,274,"2,564.22","3,464.30","3,175.22",0.52,0.48,0.49
moderate,86,183,269,86,183,269,"3,720.03","3,878.09","3,827.56",0.48,0.35,0.39
no_info,116,278,394,116,278,394,"2,708.75","3,310.17","3,133.10",0.16,0.10,0.12
rich,20,43,63,20,43,63,"1,616.05","2,438.86","2,177.65",0.20,0.23,0.22
All,310,690,1000,310,690,1000,"2,877.77","3,448.04","3,271.26",0.35,0.28,0.30
